In [1]:
import utils

In [2]:
utils.visualize_segmentation_dataset("input/dataset", 4)

Press any key to navigate. 
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1]
Found the following classes [0 1]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 3]
Found the following classes [0 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 2 3]
Found the following classes [0 1 3]
Found the following classes [0 3]
